In [1]:
import pandas as pd
import torch
import numpy as np
from scipy.stats import spearmanr
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [4]:
log = pd.read_pickle("/home/dhvc/projetos/backup/results_full_8_255.pkl")

In [5]:
log.size

428166

In [6]:
log.tail()

,img_id,interpretability_method,label,similarity_original,similarity_perturb,hm_original,hm_perturbed,alpha,clip_model
47569,5285,game,0,0.25708,0.302979,"[[0.001281, 0.001281, 0.001281, 0.001281, 0.00...","[[0.006622, 0.006622, 0.006622, 0.006622, 0.00...",0.031373,ViT-B/16
47570,5285,rollout,0,0.25708,0.302979,"[[0.0032668242, 0.0032668242, 0.0032668242, 0....","[[0.0018243162, 0.0018243162, 0.0018243162, 0....",0.031373,ViT-B/16
47571,5285,surgery,0,0.25708,0.302979,"[[0.2059582, 0.2059582, 0.2059582, 0.2059582, ...","[[0.27357385, 0.27357385, 0.27357385, 0.273573...",0.031373,ViT-B/16
47572,5285,m2ib,0,0.25708,0.302979,"[[0.15319468, 0.15319468, 0.15319468, 0.153194...","[[0.09657647, 0.09657647, 0.09657647, 0.096576...",0.031373,ViT-B/16
47573,5285,rise,0,0.25708,0.302979,"[[0.46606231266379944, 0.4622052583614432, 0.4...","[[0.02903688050122885, 0.04940982266915673, 0....",0.031373,ViT-B/16


In [15]:
group_name = "rise"
filtered_group = log[log['interpretability_method'] == group_name].head()
filtered_group

,img_id,interpretability_method,label,similarity_original,similarity_perturb,hm_original,hm_perturbed,alpha,clip_model
8,0,rise,9,0.297363,0.304443,"[[0.42429071337791646, 0.40814334982051115, 0....","[[0.9137669339894663, 0.9144961700907744, 0.91...",0.031373,ViT-B/16
17,1,rise,9,0.284180,0.299805,"[[0.4543291312897469, 0.457138849992142, 0.463...","[[0.5177132015886939, 0.4981942133942163, 0.48...",0.031373,ViT-B/16
26,2,rise,9,0.259277,0.284668,"[[0.5149611399749239, 0.5117091311486501, 0.50...","[[0.5785305843953101, 0.567606061554882, 0.554...",0.031373,ViT-B/16
35,3,rise,9,0.269531,0.287598,"[[0.3874789775628778, 0.3784433900502644, 0.37...","[[0.46758284331061556, 0.459774784394151, 0.45...",0.031373,ViT-B/16
44,4,rise,9,0.257324,0.295654,"[[0.4447333065751255, 0.4442771022778306, 0.43...","[[0.39263161248993983, 0.3876795616263061, 0.3...",0.031373,ViT-B/16


In [16]:
hm_original_values = []
hm_perturbed_values = []

for item in log[log['interpretability_method'] == group_name].head().iterrows():
    for tensor in item[1]['hm_original']:
        # Verificar se tensor é um torch.Tensor antes de usar .cpu()
        if isinstance(tensor, torch.Tensor):
            hm_original_values.extend(tensor.flatten().cpu().numpy())  # Para tensor do PyTorch
        else:
            hm_original_values.extend(tensor.flatten())  # Para arrays numpy
    
    for tensor in item[1]['hm_perturbed']:
        # Verificar se tensor é um torch.Tensor antes de usar .cpu()
        if isinstance(tensor, torch.Tensor):
            hm_perturbed_values.extend(tensor.flatten().cpu().numpy())  # Para tensor do PyTorch
        else:
            hm_perturbed_values.extend(tensor.flatten())  # Para arrays numpy
    spearman_corr, _ = spearmanr(hm_original_values, hm_perturbed_values)
    print(f"Spearman rank correlation for {group_name}: {spearman_corr}")

Spearman rank correlation for rise: 0.15440133031165781
Spearman rank correlation for rise: 0.008282607287117964
Spearman rank correlation for rise: 0.07124092412276853
Spearman rank correlation for rise: 0.04421261205514922
Spearman rank correlation for rise: -0.015594166156104677


In [ ]:
def calculate_spearman_for_group(group):
    hm_original_values = []
    hm_perturbed_values = []

    for i, row in group.iterrows():
        for tensor in row['hm_original']:
            # Verificar se tensor é um torch.Tensor antes de usar .cpu()
            if isinstance(tensor, torch.Tensor):
                hm_original_values.extend(tensor.flatten().cpu().numpy())  # Para tensor do PyTorch
            else:
                hm_original_values.extend(tensor.flatten())  # Para arrays numpy
    
        for tensor in row['hm_perturbed']:
            # Verificar se tensor é um torch.Tensor antes de usar .cpu()
            if isinstance(tensor, torch.Tensor):
                hm_perturbed_values.extend(tensor.flatten().cpu().numpy())  # Para tensor do PyTorch
            else:
                hm_perturbed_values.extend(tensor.flatten())  # Para arrays numpy

    spearman_corr, _ = spearmanr(hm_original_values, hm_perturbed_values)
    return spearman_corr

int_method_list =  ["selfattn", "gradcam", "maskclip", "eclip", "game", "rollout", "surgery", "m2ib", "rise"]

for methd in int_method_list:
    filtered_group = log[log['interpretability_method'] == methd]
    spearman_corr = calculate_spearman_for_group(filtered_group)
    print(f"Spearman rank correlation for {methd}: {spearman_corr}")


In [14]:
def topk_intersection(map1, map2, k):
    flat1 = map1.flatten()
    flat2 = map2.flatten()
    
    # Encontrar os índices dos top-K valores
    topk_indices_1 = torch.topk(flat1.clone().detach(), k).indices
    topk_indices_2 = torch.topk(flat2.clone().detach(), k).indices

    # Converter os índices para conjuntos
    set1 = set(topk_indices_1.tolist())
    set2 = set(topk_indices_2.tolist())    
    
    # Calcular a interseção entre os conjuntos
    intersection = set1.intersection(set2)
    
    return len(intersection), len(intersection) / k

# Valor de k (ajuste conforme necessário)
k = 100

# Filtrar o dataframe para os grupos desejados
group_names = ["selfattn", "gradcam", "maskclip", "eclip", "game", "rollout", "surgery", "m2ib", "rise"]

# Iterar sobre os grupos e calcular a interseção Top-K para cada um
for group_name in group_names:
    filtered_group = log[log['interpretability_method'] == group_name]
    
    # Variáveis para calcular médias
    total_intersection_count = 0
    total_intersection_ratio = 0
    count = 0
    
    # Iterar sobre as linhas do grupo filtrado
    for idx, row in filtered_group.iterrows():
        # Converter para tensores
        map1 = torch.tensor([[t.item() for t in inner] for inner in row["hm_original"]])
        map2 = torch.tensor([[t.item() for t in inner] for inner in row["hm_perturbed"]])
        
        # Calcular a interseção
        inter_count, inter_ratio = topk_intersection(map1, map2, k)
        
        # Agregar os resultados para a média
        total_intersection_count += inter_count
        total_intersection_ratio += inter_ratio
        count += 1
    
    # Calcular a média para o grupo
    average_intersection_count = total_intersection_count / count
    average_intersection_ratio = total_intersection_ratio / count
    
    # Imprimir o resultado de Top-K para o grupo
    print(f"Top-{k} intersection count for {group_name}: {average_intersection_count} with ratio: {average_intersection_ratio:.3f}")


Top-100 intersection count for selfattn: 100.0 with ratio: 1.000
Top-100 intersection count for gradcam: 100.0 with ratio: 1.000
Top-100 intersection count for maskclip: 100.0 with ratio: 1.000
Top-100 intersection count for eclip: 100.0 with ratio: 1.000
Top-100 intersection count for game: 12.409761634506243 with ratio: 0.124
Top-100 intersection count for rollout: 16.52137722285282 with ratio: 0.165
Top-100 intersection count for surgery: 11.93435489973515 with ratio: 0.119
Top-100 intersection count for m2ib: 13.14472190692395 with ratio: 0.131
Top-100 intersection count for rise: 0.25009458948164964 with ratio: 0.003
